In [1]:
import pandas as pd
import numpy as np
import operator

# Execise 1

## <span style="color:blue"> Part 1 </span>

In [2]:
def sgm(t):
    return 1 / (1 + np.exp(-t))

In [3]:
def data_gen(n, d, sigma, w, b):
    np.random.seed(0)
    X = np.random.uniform(-1, 1, (n, d))
    #np.random.seed(0)
    u = np.random.rand(n,1)
    y = []
   
    for i in range(n):
        t = (np.dot(np.transpose(w), X[i]) + b) / sigma
        if u[i][0] <= sgm(t):
            y.append(1)
        else:
            y.append(-1)
    y = np.array(y)
    return X, y

In [4]:
n = 6000
d = 2
sigma = 1
b = 0
# w
temp1 = [1]
temp2 = [0] * (d-1)
w = temp1 + temp2
w = np.array(w)

In [5]:
X, y = data_gen(n, d, sigma, w, b)

In [6]:
X[:5]

array([[ 0.09762701,  0.43037873],
       [ 0.20552675,  0.08976637],
       [-0.1526904 ,  0.29178823],
       [-0.12482558,  0.783546  ],
       [ 0.92732552, -0.23311696]])

In [7]:
y[:5]

array([-1, -1,  1, -1,  1])

## <span style="color:blue"> Part 2 </span>

In [8]:
def  Bayes(X, y, w, b):
    n = len(X)
    correct = 0
    for i in range(n):
        # predict
        y_hat = 0
        if np.dot(np.transpose(w), X[i]) +b >= 0:
            y_hat = 1
        else:
            y_hat = -1
        # check correctness
        if (y_hat == y[i]):
            correct += 1
    return (n - correct) / n

In [9]:
# n = 6000
Bayes(X, y, w, b)

0.361

The Bayes error of n = 6000 is 0.361.

## <span style="color:blue"> Part 3 </span>

### split the dataset into training samples and test samples

split the generated dataset into n/2 training samples and n/2 test samples

In [10]:
# n = 6000
# each has 3000 smaples
X_train = X[0: 3000] 
y_train = y[0: 3000]
X_test = X[3000: 6000]
y_test = y[3000: 6000]

### knn classifier

$\textbf{l1_Distance}$ return the Manhattan Distance between two samples $x_{1}$ and $x_{2}$ ie 1-norm

In [11]:
def l1_Distance(x1, x2):
     return np.linalg.norm(x1 - x2, 1)

$\textbf{l2_Distance}$ return the Euclidean Distance between two samples $x_{1}$ and $x_{2}$ ie 2-norm

In [12]:
def l2_Distance(x1, x2):
     return np.linalg.norm(x1 - x2)

$\textbf{findNearestNeighbors}$ collect the k most nearest points in training set for a given test point

In [13]:
def findNearestPoints(trainX, testPointX, k, dist):
    nearest_points = []
    distances = []
    n = len(trainX)
    for i in range(n):
        distance = 0
        if (dist == "l1"):
            distance = l1_Distance(trainX[i], testPointX)
        else: 
            distance = l2_Distance(trainX[i], testPointX)
        distances.append((trainX[i], distance, i))
        
    distances.sort(key=operator.itemgetter(1))
    
    # first k points => smallest distance => nearest
    for  i in range(k):
        nearest_points.append([distances[i][0],distances[i][2]])
        
    return nearest_points

$\textbf{predictByNearestPoints}$ calculates the $\hat{y_{i}}$ by taking the majority vote among each nearest points as predicted category

In [14]:
def predictByNearestPoints(nearest_points, trainY):
    pos_votes = 0
    neg_votes = 0
    for i in range(len(nearest_points)):
        idx = nearest_points[i][1]
        if (trainY[idx] == 1):
            pos_votes += 1
        else :
            neg_votes += 1
    if pos_votes > neg_votes:
        return 1
    else:
        return -1

$\textbf{predictionError}$ calculates the prediction errors by taking a ratio of total correct prediction out of all predictions

In [15]:
def predictionError(predictions, trueValues):
    correct = 0
    total = len(predictions)
    for i in range(total):
        if (predictions[i] == trueValues[i]):
            correct += 1
    #return round((correct/float(total) * 100.0, 3)
    return (total - correct)/float(total)

Note: dist is either "l1" or "l2"


$\textbf{knn}$ simulates the k-NN classifier by using either $l_{1}$ distance or $l_{2}$ distance, and returns the predictions ($\hat{testY}$) and the prediction error

In [16]:
def knn(trainX, trainY, testX, k, dist, testY):
    size = len(testX)
    predictions = [] # list of testYhat
    for i in range(size):
        #print(i)
        nearest_points = findNearestPoints(trainX, testX[i], k, dist) #list of [point, idx]
        predict = predictByNearestPoints(nearest_points, trainY)
        predictions.append(predict)
    errors = predictionError(predictions, testY)
    return predictions, errors
    

### knn classifier using $l_{2}$ distance

In [17]:
dist = "l2"
# k = 1
testYhat_k1, errors_k1 = knn(X_train, y_train, X_test, 1, dist, y_test)

In [18]:
# k = 3
testYhat_k3, errors_k3 = knn(X_train, y_train, X_test, 3, dist, y_test)

In [19]:
# k = 5
testYhat_k5, errors_k5 = knn(X_train, y_train, X_test, 5, dist, y_test)

In [20]:
errors = [errors_k1, errors_k3, errors_k5]
print(errors)

[0.4513333333333333, 0.43333333333333335, 0.41133333333333333]


When k = 1, k-NN error is 0.4513333333333333.

When k = 3, k-NN error is 0.43333333333333335.

When k = 5, k-NN error is 0.41133333333333333.

## <span style="color:blue"> Part 4 </span>

### knn classifier using $l_{1}$ distance

In [21]:
dist = "l1"
# k = 1
testYhat_k1, errors_k1 = knn(X_train, y_train, X_test, 1, dist, y_test)

In [22]:
# k = 3
testYhat_k3, errors_k3 = knn(X_train, y_train, X_test, 3, dist, y_test)

In [23]:
# k = 5
testYhat_k5, errors_k5 = knn(X_train, y_train, X_test, 5, dist, y_test)

In [24]:
errors = [errors_k1, errors_k3, errors_k5]
print(errors)

[0.4543333333333333, 0.43366666666666664, 0.41533333333333333]


When k = 1, k-NN error is 0.4543333333333333.

When k = 3, k-NN error is 0.43366666666666664.

When k = 5, k-NN error is 0.41533333333333333.

## <span style="color:blue"> Part 5 </span>

Compare Bayes error and 1-NN error for differnt $\sigma$.

For each $\sigma$:

1. generate dataset as part 1

2. split training set and test set

3. compute Bayes error (part 2) and 1-NN error(part 3) repectively

In [25]:
def Bayes_vs_1nn(n, d, sigmas, w, b):
    errors = []
    for sigma in sigmas:
        #generate dataset
        X, y = data_gen(n, d, sigma, w, b)
        # split the generated dataset into n/2 training samples and n/2 test samples
        # n = 6000
        X_train = X[0: 3000] 
        y_train = y[0: 3000]
        X_test = X[3000: n]
        y_test = y[3000: n]
        # Bayes error
        bayes_error = Bayes(X, y, w, b)
        # 1-NN error
        testYhat, error = knn(X_train, y_train, X_test, 1, "l2", y_test)
        
        print("sigma = " + str(sigma) +", bayes error = " + str(bayes_error) + ",1-NN error = " + str(error))
        errors.append((bayes_error, error))
    return errors

In [26]:
sigmas = [0.01, 0.1, 1, 10]
# n = 6000
errors = Bayes_vs_1nn(n, d, sigmas, w, b)

sigma = 0.01, bayes error = 0.0065,1-NN error = 0.012333333333333333
sigma = 0.1, bayes error = 0.066,1-NN error = 0.098
sigma = 1, bayes error = 0.361,1-NN error = 0.4513333333333333
sigma = 10, bayes error = 0.4726666666666667,1-NN error = 0.48


From the report above:

1. As $\sigma$ increases, bayes error and 1-NN error both increase.

2. For each $\sigma$, bayes error is always lower than  1-NN error.

3. For a large $\sigma$, both the errors have an upper bounf of 50%. For a small $\sigma$, 1-NN error is almost twice the Bayes error.